In [30]:
from ipywidgets import widgets, Layout
from IPython.display import display

In [ ]:
#credits
#styling help from https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html

In [43]:
#create widgets
style = {'description_width': 'initial'}
keyIn = widgets.Textarea(placeholder='here', description='Enter your plaintext:', disabled=False, style=style)
dropdown = widgets.Dropdown(options=['RSA', 'Diffe-Hellman','Elliptic curve'], value='RSA', 
                 description='Choose an encryption method:', disabled=False, style=style)
submit = widgets.Button(description='Submit', disabled=False)
output = widgets.Output(layout={'border':'1px solid black'})

#do calculations for widgets
with output:
    #if (dropdown=='RSA') {
    #output.clear_output()
    #rsa algo
    #}
    #else if (dropdown == 'Diffe-Hellman') {
    #output.clear_output()
    #dh algo
    #}
    #else{
    #output.clear_output()
    #elliptic algo
    #}

#display widgets
display(keyIn)
display(dropdown, submit)
display(output)

Textarea(value='', description='Enter your plaintext:', placeholder='here', style=DescriptionStyle(description…

Dropdown(description='Choose an encryption method:', options=('RSA', 'Diffe-Hellman', 'Elliptic curve'), style…

Button(description='Submit', style=ButtonStyle())